### Genderizer
Classificateur de genre : Utilisé dans le contexte de l'analyse linguistique ou du traitement automatique des langues.
Utiliser Pytorch


Feuille de route:
- faire un état de l'art rapide sur le web
    - Nooj
    - HuggingFace
- choisir un dictionaire gratuit open source > Dictionnaire de l’Académie française de 1932-35 (8e édition) - GLP3
- créer une base de données de noms communs étiquetés par genre
- faire une analyse des données

In [1]:
import os
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import urllib.request
import re
from IPython.display import display, HTML

In [11]:
# importer la source au format Mobipocket
url = "https://www.ebooksgratuits.com/newsendbook.php?id=1609&format=mp"
urllib.request.urlretrieve(url, "aca.prc")

print("Download complete.")

Download complete.


In [8]:
pip install mobi

Note: you may need to restart the kernel to use updated packages.


In [2]:
!mobiunpack -r aca.prc ./


KindleUnpack v0.82
   Based on initial mobipocket version Copyright © 2009 Charles M. Hannum <root@ihack.net>
   Extensive Extensions and Improvements Copyright © 2009-2014 
       by:  P. Durrant, K. Hendricks, S. Siebert, fandrieu, DiapDealer, nickredding, tkeo.
   This program is free software: you can redistribute it and/or modify
   it under the terms of the GNU General Public License as published by
   the Free Software Foundation, version 3.
Unpacking Book...
Completed


In [14]:
# prend environ 5 minutes

# Read the HTML content from the file
with open('./mobi7/book.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Define the regular expression patterns to match <idx:orth> and <idx:entry> tags
pattern_orth = re.compile(r'<idx:orth value="[^"]*">')
# pattern_entry = re.compile(r'<idx:entry>.*?</idx:entry>', re.DOTALL)
pattern_entry = re.compile(r'</idx:entry>.*?<mbp:pagebreak/>', re.DOTALL)

# Find all matches for <idx:orth> and <idx:entry> tags
orth_matches = pattern_orth.findall(html_content)
entry_matches = pattern_entry.findall(html_content)


In [15]:

print("Number of <idx:orth> tags:", len(orth_matches))
print("Number of <idx:entry> tags:", len(entry_matches))

Number of <idx:orth> tags: 31933
Number of <idx:entry> tags: 31933


In [16]:

# Print the last extracted tags
print("Extracted <idx:orth> tags:")
for tag in orth_matches[-4:]:
    print(tag)

print("\nExtracted <idx:entry> tags:")
for tag in entry_matches[-4:]:
    print(tag)

Extracted <idx:orth> tags:
<idx:orth value="ZOUAVE">
<idx:orth value="ZUT">
<idx:orth value="ZYGOMA">
<idx:orth value="ZYGOMATIQUE">

Extracted <idx:entry> tags:
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZOUAVE</b></font></h2><br/> <font color="green">n. m.</font><br/>Soldat d'un corps d'infanterie servant pour la plus grande partie en Afrique du Nord. <p> <i>Zouaves pontificaux,</i> Corps de volontaires constitué en 1860 par le général de Lamoricière pour la défense des États pontificaux. <mbp:pagebreak/>
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZUT</b></font></h2><br/> <font color="green">interj.</font><br/>Exclamation familière pour exprimer l'impatience, le dépit, pour envoyer les gens au diable et signifier qu'on ne veut plus rien entendre. <mbp:pagebreak/>
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZYGOMA</b></font></h2><br/> <font color="green">n. m.</font><br/><dom>T. d'Anatomie</dom><br/>Os de la pommette de la joue. <mbp:pagebreak/>
</idx:entry>
<h2><font

In [18]:
import os
import sqlite3

db_file = "acad_pocket.db"         # SQLite database file



# === STEP 2: INITIALIZE SQLITE DATABASE ===
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create a table for words and definitions
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dictionary (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        word TEXT,
        definition TEXT
    )
""")


# Insert words and definitions into the database
for word, definition in zip(orth_matches, entry_matches):

    word = re.search(r'value="([^"]*)"', word).group(1)
    definition = definition.replace('</idx:entry>', '').replace('<mbp:pagebreak/>', '')
    
    try:
        cursor.execute("INSERT INTO dictionary (word, definition) VALUES (?, ?)", (word, definition))
        
    except sqlite3.IntegrityError:
        pass  # Ignore duplicate words

# === STEP 4: FINALIZE ===
conn.commit()
conn.close()

print(f"Dictionary has been saved to {db_file}")

Dictionary has been saved to acad_pocket.db


In [59]:
def acad_definition(word):
    # Connect to the SQLite database
    conn = sqlite3.connect('acad_pocket.db')

    # Write a query to fetch data
    query = """
    SELECT * FROM dictionary
    WHERE word = '{}'
    """.format(word.upper())

    # Use pandas to read the data into a DataFrame
    df_ = pd.read_sql_query(query, conn)

    # Close the connection
    conn.close()

    # Check if the DataFrame is empty
    if df_.empty:
        print("Absent du dictionnaire de l'Académie Française de 1935")
    else:
        # Display the DataFrame with full column content
        pd.set_option('display.max_colwidth', None)
        display(df_)
        pd.reset_option('display.max_colwidth')
        for ht in df_.definition:
            # Display the HTML content
            display(HTML(ht))

    del df_, query, conn

In [60]:
acad_definition("tour")

id  word  \
0  30201  TOUR   
1  30202  TOUR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

---

In [2]:
def list_tags(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    list_balises = set()
    # Iterate over all tags in the HTML
    for tag in soup.find_all(True):
        list_balises.add(tag.name)

    return list_balises

In [ ]:
# Read the HTML content from the file - environ 5 minutes
with open('./mobi7/book.html', 'r', encoding='utf-8') as file:
    html_content = file.read()
print(list_tags(html_content))

['a', 'b', 'body', 'br', 'div', 'dom', 'fm', 'font', 'guide', 'h2', 'h3', 'head', 'html', 'i', 'idx:entry', 'idx:orth', 'img', 'mbp:frameset', 'mbp:pagebreak', 'meta', 'p', 'reference', 'renv', 'sup']


In [24]:
with sqlite3.connect('acad_pocket.db') as conn:
    acad = pd.read_sql_query("SELECT * FROM dictionary", conn)
acad.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31933 entries, 0 to 31932
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          31933 non-null  int64 
 1   word        31933 non-null  object
 2   definition  31933 non-null  object
dtypes: int64(1), object(2)
memory usage: 748.6+ KB


In [33]:
# Add a new column to acad that includes the text between '<font color="green">' and '</font>' from the definition column
acad['cat_list'] = acad['definition'].apply(lambda x: re.findall(r'<font color="green">(.*?)</font>', x))

In [34]:
acad['cat'] = acad['cat_list'].apply(lambda x: x[0] if x else None)


In [37]:
cat_list = []
for list in acad['cat_list']:
    if list:
        for item in list:
            if item not in cat_list:
                cat_list.append(item)
cat_list.sort()
print(cat_list)

['<i>(Je couvre ; nous couvrons. Je couvrais. Je couvris. Je couvrirai. Couvre. Que je couvre. Que je couvrisse. Couvrant. Couvert.)</i> v. tr.', 'Adjectif et pronom des deux genres', 'Adjectif possessif masculin', 'Adverbe', 'Conjonction', 'Conjonction alternative', 'Interj.', 'Interjection', 'Interjection familière', 'Interjection populaire', 'Interjection qui exprime la douleur.', 'Loc. adv.', 'Loc. adverb.', 'Loc. interj.', 'Loc. prép.', 'Locution', 'Locution adverbiale', 'Locution adverbiale empruntée du latin.', 'Locution conjonctive', 'Locution empruntée du latin.', 'Locution familière empruntée du latin.', 'Locution latine', 'Locution prépositive', 'Nom et adjectif de nombre cardinal invariable', 'Onomatopée', 'Participe présent', 'Participe présent employé substantivement', 'Particule', 'Particule adverbiale', 'Particule affirmative', "Particule d'origine latine", 'Particule grecque', 'Particule négative', 'Pron. dém.CELUI, m. sing; CELLE, f. sing.; CEUX, m. plur.; CELLES, f. 

In [62]:
cat_0_list = []
for item in acad['cat']:
    if item:
        if item not in cat_0_list:
            cat_0_list.append(item)
cat_0_list.sort()
print("\n".join(cat_0_list))

<i>(Je couvre ; nous couvrons. Je couvrais. Je couvris. Je couvrirai. Couvre. Que je couvre. Que je couvrisse. Couvrant. Couvert.)</i> v. tr.
Adjectif et pronom des deux genres
Adjectif possessif masculin
Adverbe
Conjonction
Conjonction alternative
Interj.
Interjection
Interjection familière
Interjection populaire
Interjection qui exprime la douleur.
Loc. adv.
Loc. adverb.
Loc. interj.
Loc. prép.
Locution
Locution adverbiale
Locution adverbiale empruntée du latin.
Locution empruntée du latin.
Locution familière empruntée du latin.
Locution latine
Nom et adjectif de nombre cardinal invariable
Onomatopée
Participe présent
Participe présent employé substantivement
Particule
Particule adverbiale
Particule affirmative
Particule d'origine latine
Particule grecque
Particule négative
Pron. dém.CELUI, m. sing; CELLE, f. sing.; CEUX, m. plur.; CELLES, f. plur.
Pronom de la troisième personne, des deux genres
Pronom invariable
Pronom masculin indéfini
Pronom personnel
Pronom personnel des deux ge

In [66]:
cat_noms = [ cat for cat in cat_0_list if 'n.' in cat.split() or 'nom' in cat.split() ]
print("Nombre de catégories de noms:", len(cat_noms))
print("\n".join(cat_noms))

Nombre de catégories de noms: 28
adj. des deux genres et n. m.
adj. et n.
adj. et n. des deux genres
adj. et n. f.
adj. et n. m.
adj. et nom numéral cardinal des deux genres
n.
n. des deux genres
n. et adj.
n. f.
n. f. coll.
n. f. et adj. des deux genres
n. f. et n. m.
n. f. pl.
n. f. pluriel
n. invariable
n. m.
n. m. coll.
n. m. et f.
n. m. et f. pl.
n. m. inv.
n. m. invariable
n. m. pl.
n. m. plur.
n. m. sans pluriel
n. m., Expression empruntée du latin.
n. m., Locution empruntée du latin.
nom et adj. numéral cardinal invariable


In [61]:
acad_definition("courir")

id    word  \
0  7473  COURIR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              